In [17]:
import streamlit as st
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
load_dotenv()


True

In [18]:
embedding=OllamaEmbeddings(model="llama2:7b")
api_key=os.getenv("GROQ_API")

llm=ChatGroq(groq_api_key=api_key,model_name='Llama3-8b-8192')

In [24]:
name="title.pdf"
def download_pdf(url, output_path=f"./storage/{name}"):
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded PDF to {output_path}")
        return output_path
    else:
        raise Exception(f"Failed to download PDF: {response.status_code}")

pdf_path = download_pdf("https://www.dropbox.com/scl/fi/tqc5bz8ete0j8n6ihbntr/AI_ML-SD-Intern.pdf?rlkey=netrg0jdt1jan7j9ggief6tl0&dl=1")


Downloaded PDF to ./storage/title.pdf


In [11]:
pdf_path = "./temp.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [ ]:
persist_directory = './testing_db'

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

vectorstore.persist()

C:\Users\mukka\AppData\Local\Temp\ipykernel_18992\1618585710.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [14]:
retriever = vectorstore.as_retriever()
contextualized_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided context."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualized_q_prompt)

system_prompt = (
    "You are a helpful assistant that answers questions based on the provided context. "
    "If you don't know the answer, just say that you don't know. "
    "You can also ask for clarification if needed. Answer concisely.\n\n{context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])


In [16]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# 6. Session and Message History
session_id = "default_session"
store = {}
def get_session_history(session: str) -> BaseChatMessageHistory:
    if session not in store:
        store[session] = ChatMessageHistory()
    return store[session]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

# 7. Interactive Chat Loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    session_history = get_session_history(session_id)
    response = conversational_rag_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )
    print("Assistant:", response["answer"])
    print("\n--- Chat History ---")
    for msg in session_history.messages:
        role = "You" if msg.type == "human" else "Assistant"
        print(f"{role}: {msg.content}")
    print("---------------------\n")

Assistant: According to the job description, the stipend for the Software Developer Intern position is up to 50,000 per month.

--- Chat History ---
You: i want to know the stipend
Assistant: According to the job description, the stipend for the Software Developer Intern position is up to 50,000 per month.
---------------------

